# ChessML Netz

This neural network based on PyTorch will try to classifiy chess pieces from photos taken from the top of a chessboard.

In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import random

from torchvision import transforms
from PIL import Image
from os import listdir

## Defining all possible classes

bb = Black Bishop
bk = Black King
bn = Black Knight
bp = Black Pawn
bq = Black Queen
br = Black Rook
...

In [ ]:
CLASSES = ["bb", "bk", "bn", "bp", "bq", "br", "wb", "wk", "wn", "wp", "wq", "wr", "empty"]

## Define a normalization function for the analyzed data

The values are recommended by a PyTorch tutorial (https://youtu.be/32lHVbT09h8).

In [ ]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    normalize
])

## Defining the Neural Network

In [ ]:
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        # Defining the convolutional layers of the net
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv3 = nn.Conv2d(20, 30, kernel_size=5)
        
        # Defining dropout layer to prevent from memorizing
        self.conv_dropout = nn.Dropout2d()
        
        # Defining the fully connected layers of the net
        self.fc1 = nn.Linear(4320, 1000) # 30*12*12 = 4320
        self.fc2 = nn.Linear(1000, 13)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x) # Relu because it's famous

        x = self.conv2(x)
        x = self.conv_dropout(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = x.view(-1, 4320) # Convert 2d data to 1d
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        
        return torch.sigmoid(x)


## Training

In [ ]:
def train(model, epoch, train_data, optimizer):
    model.train()
    batch_id = 0
    
    for data, target in train_data:
        target = torch.Tensor(target)
        
        # Activate cuda support if available
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
            
        optimizer.zero_grad()
        out = model(data)
        criterion = F.binary_cross_entropy
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_id * len(data), len(train_data) * len(data), 100. * batch_id / len(train_data), loss.item()
        ))
        batch_id = batch_id + 1

## Evaluation

In [ ]:
def eval(model, path):
    model.eval()


## Testing

In [ ]:
def test(model, path):
    model.eval()
    files = os.listdir(path)
    file = random.choice(files)
    img = Image.open(os.path.join(path, file))
    img_eval_tensor = transform(img)
    img_eval_tensor.unsqueeze_(0)
    if torch.cuda.is_available():
        img_eval_tensor = img_eval_tensor.cuda()
    out = model(img_eval_tensor)
    print(out.data.max(1, keepdim=True)[1])
    img.show()
    x = input("")


At first all available files are added to a file list. In addition the label of the file will be generated and added to the list.

In [ ]:
def read_training_files(path):
    file_list = []
    for d in os.listdir(path):
        if os.path.isdir(os.path.join(path, d)):
            for f in os.listdir(os.path.join(path, d)):
                file_list.append([os.path.join(path, d, f), hot_encode_label(d)])
            break
    return file_list


In [ ]:
def generate_training_tensors(file_list, batch_size):
    train_data_list = []
    train_data = []
    label_list = []
    files_count = len(file_list)
    for i in range(files_count):
        file = random.choice(file_list)
        file_list.remove(file)
        img = Image.open(file[0])  # Index 0: filename, Index 1: label
        img_tensor = transform(img)
        train_data_list.append(img_tensor)
        label_list.append(file[1])
        if len(train_data_list) >= batch_size:
            train_data.append((torch.stack(train_data_list), label_list))
            train_data_list = []
            label_list = []
            print("Loaded batch", len(train_data), "of", int(files_count / batch_size))
            print("Percentage Done:", len(train_data) / int(files_count / batch_size) * 100, "%")
    return train_data


In [ ]:
def hot_encode_label(label):
    index = CLASSES.index(label)
    vector = np.zeros(len(CLASSES), np.int8)
    vector[index] = 1
    return vector


In [ ]:
def main():
    file_list = read_training_files("data/train_augmented")
    train_data = generate_training_tensors(file_list, 64)

    model = ChessNet()
    # Activate cuda support if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Check if model is already available
    if os.path.isfile("model/chess-net.pt"):
        model = torch.load("model/chess-net.pt")

    # Defining the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    # optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

    for epoch in range(1):
        train(model, epoch, train_data, optimizer)
        torch.save(model, "model/chess-net.pt")
    print("Training of the neuroal network done.")


In [ ]:
if __name__ == "__main__":
    main()
